In [9]:
"""
Combine Fed Communications Data
Combines minutes, statements, transcripts, speeches, and press conferences
into a single dataset with standardized variables.
"""

from google.colab import drive
drive.mount("/content/drive", force_remount=True)

import pandas as pd
import numpy as np
from datetime import datetime
import os

# ============================================================================
# CONFIGURATION
# ============================================================================

BASE_DIR = '/content/drive/MyDrive/FedComs'
OUTPUT_DIR = f'{BASE_DIR}/SummaryStats'

# Input files
INPUT_FILES = {
    'minutes': f'{BASE_DIR}/Minutes/minutes_content.csv',
    'statements': f'{BASE_DIR}/Statements/statement_content.csv',
    'transcripts': f'{BASE_DIR}/Transcripts/transcripts_content.csv',
    'speeches': f'{BASE_DIR}/Speeches/speeches_content.csv',
    'presscon': f'{BASE_DIR}/PressConf/press_conferences_content.csv'
}

# ============================================================================
# SPEAKER NAME STANDARDIZATION
# ============================================================================

def standardize_speaker_name(name):
    """
    Convert full names to format: firstinitiallastname (e.g., 'jpowell')
    Handle various name formats and return '.' for missing values.
    """
    if pd.isna(name) or name == '' or str(name).strip() == '':
        return '.'

    name = str(name).strip()

    # Handle special cases
    if name.lower() in ['other', 'fomc', '.']:
        return name.lower() if name.lower() != '.' else '.'

    # Remove common titles and suffixes
    name = name.replace('Jr.', '').replace('Jr', '').replace('Sr.', '').replace('Sr', '')
    name = name.replace('Dr.', '').replace('Dr', '').replace('Mr.', '').replace('Ms.', '')

    # Split name into parts
    parts = name.split()

    if len(parts) == 0:
        return '.'
    elif len(parts) == 1:
        # Just last name
        return parts[0].lower()
    else:
        # First initial + last name
        first_initial = parts[0][0].lower()
        last_name = parts[-1].lower()
        return f"{first_initial}{last_name}"

# ============================================================================
# INSTITUTION AND PRESIDENT MAPPINGS
# ============================================================================

# Regional bank mapping
REGIONAL_BANKS = {
    'boston': 'Boston',
    'new york': 'New York',
    'philadelphia': 'Philadelphia',
    'cleveland': 'Cleveland',
    'richmond': 'Richmond',
    'atlanta': 'Atlanta',
    'chicago': 'Chicago',
    'st. louis': 'St. Louis',
    'minneapolis': 'Minneapolis',
    'kansas city': 'Kansas City',
    'dallas': 'Dallas',
    'san francisco': 'San Francisco'
}

# President who appointed each Fed official (Board members only)
# Format: standardized_name: (president, start_year, end_year)
APPOINTING_PRESIDENT = {
    # Chairs
    'agreenspan': ('Reagan', 1987, 2006),
    'bbernanke': ('Bush', 2006, 2014),
    'jyellen': ('Obama', 2014, 2018),
    'jpowell': ('Trump', 2018, None),

    # Governors (major ones from 2000-present)
    'sbies': ('Clinton', 2001, 2007),
    'rjr': ('Clinton', 1997, 2006),  # Roger Ferguson
    'rferguson': ('Clinton', 1997, 2006),
    'molson': ('Clinton', 2001, 2006),
    'swarsh': ('Clinton', 1997, 2002),
    'egramlich': ('Clinton', 1997, 2005),
    'lmeyer': ('Clinton', 1996, 2002),
    'eguernsey': ('Bush', 2006, 2008),
    'rkreszner': ('Bush', 2006, 2009),
    'kmishkin': ('Bush', 2006, 2008),
    'fmishkin': ('Bush', 2006, 2008),
    'kwarsh': ('Bush', 2006, 2011),
    'dkohn': ('Clinton', 2002, 2010),
    'etarullo': ('Obama', 2009, 2017),
    'dduke': ('Obama', 2008, 2013),
    'sraskin': ('Obama', 2010, 2014),
    'jstein': ('Obama', 2012, 2014),
    'jyellen': ('Obama', 2010, 2018),  # Also as Governor before Chair
    'jpowell': ('Obama', 2012, None),  # Also as Governor before Chair
    'lbrainard': ('Obama', 2014, 2024),
    'rquarles': ('Trump', 2017, 2022),
    'mbowman': ('Trump', 2018, None),
    'rquarles': ('Trump', 2017, 2022),
    'mclarida': ('Trump', 2018, 2022),
    'cwaller': ('Trump', 2020, None),
    'ljefferson': ('Biden', 2022, None),
    'pcook': ('Biden', 2022, None),
    'mkugler': ('Biden', 2023, None),
    'abarr': ('Biden', 2022, None),

    # Add more as needed - this covers most major officials 2000-2025
}

# ============================================================================
# ROLE DETERMINATION
# ============================================================================

def determine_role(speaker, role_field, date, com_type):
    """
    Determine role: Chair, Governor, Regional President, FOMC, or Other
    """
    # For statements and minutes, always FOMC
    if com_type in ['statements', 'minutes']:
        return 'FOMC'

    # For press conferences with "Other"
    if com_type == 'presscon' and speaker == 'other':
        return 'Other'

    # If role field exists, use it
    if pd.notna(role_field) and role_field != '':
        role_str = str(role_field).strip()

        # Check for Chair
        if 'chair' in role_str.lower() and 'vice' not in role_str.lower():
            return 'Chair'

        # Check for Governor (including Vice Chair)
        if 'governor' in role_str.lower() or 'vice chair' in role_str.lower():
            return 'Governor'

        # Regional banks
        if any(bank in role_str.lower() for bank in REGIONAL_BANKS.keys()):
            return 'Regional President'

    # Fallback: determine chair by date if speaker is standardized chair name
    if pd.notna(date):
        try:
            date_obj = pd.to_datetime(date)

            # Known Fed Chairs and their tenures
            if speaker in ['agreenspan', 'greenspan']:
                if date_obj < pd.to_datetime('2006-02-01'):
                    return 'Chair'
            elif speaker in ['bbernanke', 'bernanke']:
                if pd.to_datetime('2006-02-01') <= date_obj < pd.to_datetime('2014-02-01'):
                    return 'Chair'
            elif speaker in ['jyellen', 'yellen']:
                if pd.to_datetime('2014-02-01') <= date_obj < pd.to_datetime('2018-02-05'):
                    return 'Chair'
            elif speaker in ['jpowell', 'powell']:
                if date_obj >= pd.to_datetime('2018-02-05'):
                    return 'Chair'
        except:
            pass

    return '.'

def determine_institution(speaker, role_field, role):
    """
    Determine institution: Board or specific Regional Bank
    Returns '.' for FOMC, Other, or missing data
    """
    # No institution for FOMC or Other
    if role in ['FOMC', 'Other'] or speaker in ['fomc', 'other', '.']:
        return '.'

    # Board members (Chair and Governors)
    if role in ['Chair', 'Governor']:
        return 'Board'

    # Regional Presidents - extract from role field
    if pd.notna(role_field) and role_field != '':
        role_str = str(role_field).strip().lower()
        for bank_key, bank_name in REGIONAL_BANKS.items():
            if bank_key in role_str:
                return bank_name

    return '.'

def determine_appointing_president(speaker, date, role):
    """
    Determine which president appointed the official (Board members only)
    Returns '.' for non-Board members or unknown
    """
    # Only for Board members
    if role not in ['Chair', 'Governor']:
        return '.'

    if speaker in ['.', 'fomc', 'other']:
        return '.'

    # Look up in mapping
    if speaker in APPOINTING_PRESIDENT:
        pres_info = APPOINTING_PRESIDENT[speaker]
        president = pres_info[0]

        # Verify date is within tenure (if date available)
        if pd.notna(date):
            try:
                date_obj = pd.to_datetime(date)
                year = date_obj.year

                start_year = pres_info[1]
                end_year = pres_info[2]

                # Check if date is within tenure
                if end_year is None:  # Still serving
                    if year >= start_year:
                        return president
                else:
                    if start_year <= year <= end_year:
                        return president
            except:
                pass

        # If no date validation, just return the president
        return president

    return '.'

# ============================================================================
# DATE PROCESSING
# ============================================================================

def process_date(date_val):
    """
    Convert date to standard format and extract year/month.
    Returns: (formatted_date, year, month) or ('.', '.', '.') if invalid
    """
    if pd.isna(date_val) or date_val == '' or str(date_val).strip() == '':
        return '.', '.', '.'

    try:
        # Parse date
        date_obj = pd.to_datetime(date_val, errors='coerce')

        if pd.isna(date_obj):
            return '.', '.', '.'

        # Format as YYYY-MM-DD
        formatted_date = date_obj.strftime('%Y-%m-%d')
        year = str(date_obj.year)
        month = str(date_obj.month)

        return formatted_date, year, month
    except:
        return '.', '.', '.'

# ============================================================================
# CONTENT VARIABLES - Define the 77 variables we want to keep
# ============================================================================

CONTENT_VARS = [
    'source_url',
    'sentences_on_labor',
    'sentences_on_inflation',
    'sentences_on_both',
    'total_sentences',
    'labor_share_of_labor_inflation_sentences',
    # Labor counts
    'labor_Employment_count',
    'labor_Unemployment_count',
    'labor_Participation_count',
    'labor_Wages_count',
    'labor_Vacancies_count',
    'labor_Quits_count',
    'labor_Layoffs_count',
    'labor_Hiring_count',
    # Inflation counts
    'inflation_Commodity_Prices_count',
    'inflation_Core_count',
    'inflation_Core_CPI_count',
    'inflation_Core_PCE_count',
    'inflation_Energy_count',
    'inflation_Food_count',
    'inflation_Goods_count',
    'inflation_Headline_count',
    'inflation_Headline_CPI_count',
    'inflation_Headline_PCE_count',
    'inflation_Housing_count',
    'inflation_Inflation_Expectations_count',
    'inflation_PPI_count',
    'inflation_Services_count',
    'inflation_Wage_Inflation_count',
    # Labor emphasis
    'labor_emphasis_Employment',
    'labor_emphasis_Unemployment',
    'labor_emphasis_Participation',
    'labor_emphasis_Wages',
    'labor_emphasis_Vacancies',
    'labor_emphasis_Quits',
    'labor_emphasis_Layoffs',
    'labor_emphasis_Hiring',
    # Inflation emphasis
    'inflation_emphasis_Commodity_Prices',
    'inflation_emphasis_Core',
    'inflation_emphasis_Core_CPI',
    'inflation_emphasis_Core_PCE',
    'inflation_emphasis_Energy',
    'inflation_emphasis_Food',
    'inflation_emphasis_Goods',
    'inflation_emphasis_Headline',
    'inflation_emphasis_Headline_CPI',
    'inflation_emphasis_Headline_PCE',
    'inflation_emphasis_Housing',
    'inflation_emphasis_Inflation_Expectations',
    'inflation_emphasis_PPI',
    'inflation_emphasis_Services',
    'inflation_emphasis_Wage_Inflation',
    # Labor share of total sentences
    'labor_share_total_sentences_Employment',
    'labor_share_total_sentences_Unemployment',
    'labor_share_total_sentences_Participation',
    'labor_share_total_sentences_Wages',
    'labor_share_total_sentences_Vacancies',
    'labor_share_total_sentences_Quits',
    'labor_share_total_sentences_Layoffs',
    'labor_share_total_sentences_Hiring',
    # Inflation share of total sentences
    'inflation_share_total_sentences_Commodity_Prices',
    'inflation_share_total_sentences_Core',
    'inflation_share_total_sentences_Core_CPI',
    'inflation_share_total_sentences_Core_PCE',
    'inflation_share_total_sentences_Energy',
    'inflation_share_total_sentences_Food',
    'inflation_share_total_sentences_Goods',
    'inflation_share_total_sentences_Headline',
    'inflation_share_total_sentences_Headline_CPI',
    'inflation_share_total_sentences_Headline_PCE',
    'inflation_share_total_sentences_Housing',
    'inflation_share_total_sentences_Inflation_Expectations',
    'inflation_share_total_sentences_PPI',
    'inflation_share_total_sentences_Services',
    'inflation_share_total_sentences_Wage_Inflation'
]

# ============================================================================
# PROCESS EACH DATA SOURCE
# ============================================================================

def process_minutes(filepath):
    """Process FOMC minutes."""
    print("\n" + "="*70)
    print("PROCESSING: MINUTES")
    print("="*70)

    df = pd.read_csv(filepath)
    print(f"Loaded {len(df)} records")

    # Create standardized columns
    df['com_type'] = 'minutes'
    df['speaker'] = 'fomc'
    df['role'] = 'FOMC'

    # Process dates
    df[['date', 'year', 'month']] = df['date'].apply(
        lambda x: pd.Series(process_date(x))
    )

    # Add institution and president (blank for FOMC)
    df['institution'] = '.'
    df['president'] = '.'

    # Get source file name
    df['source_file'] = 'minutes_content.csv'

    # Keep only needed columns
    keep_cols = ['id', 'com_type', 'speaker', 'role', 'institution', 'president',
                 'date', 'year', 'month', 'source_file']

    # Add content variables that exist
    for var in CONTENT_VARS:
        if var in df.columns:
            keep_cols.append(var)
        else:
            df[var] = '.'
            keep_cols.append(var)

    return df[keep_cols]

def process_statements(filepath):
    """Process FOMC statements."""
    print("\n" + "="*70)
    print("PROCESSING: STATEMENTS")
    print("="*70)

    df = pd.read_csv(filepath)
    print(f"Loaded {len(df)} records")

    # Create standardized columns
    df['com_type'] = 'statements'
    df['speaker'] = 'fomc'
    df['role'] = 'FOMC'
    df['source_url'] = '.'  # Statements don't have source_url

    # Process dates
    df[['date', 'year', 'month']] = df['date'].apply(
        lambda x: pd.Series(process_date(x))
    )

    # Add institution and president (blank for FOMC)
    df['institution'] = '.'
    df['president'] = '.'

    # Get source file name
    df['source_file'] = 'statement_content.csv'

    # Keep only needed columns
    keep_cols = ['id', 'com_type', 'speaker', 'role', 'institution', 'president',
                 'date', 'year', 'month', 'source_file']

    # Add content variables that exist
    for var in CONTENT_VARS:
        if var in df.columns:
            keep_cols.append(var)
        else:
            df[var] = '.'
            keep_cols.append(var)

    return df[keep_cols]

def process_transcripts(filepath):
    """Process FOMC transcripts."""
    print("\n" + "="*70)
    print("PROCESSING: TRANSCRIPTS")
    print("="*70)

    df = pd.read_csv(filepath)
    print(f"Loaded {len(df)} records")

    # Standardize speaker names
    df['speaker'] = df['official_name'].apply(standardize_speaker_name)

    # Process dates
    df[['date', 'year', 'month']] = df['date'].apply(
        lambda x: pd.Series(process_date(x))
    )

    # Store original role field for institution determination
    df['role_original'] = df['role']

    # Determine roles
    df['role'] = df.apply(
        lambda row: determine_role(row['speaker'], row['role_original'], row['date'], 'transcripts'),
        axis=1
    )

    # Determine institution
    df['institution'] = df.apply(
        lambda row: determine_institution(row['speaker'], row['role_original'], row['role']),
        axis=1
    )

    # Determine appointing president
    df['president'] = df.apply(
        lambda row: determine_appointing_president(row['speaker'], row['date'], row['role']),
        axis=1
    )

    # Rename id column
    df = df.rename(columns={'transcript_id': 'id'})

    # Create standardized columns
    df['com_type'] = 'transcripts'
    df['source_url'] = '.'  # Transcripts don't have source_url
    df['source_file'] = 'transcripts_content.csv'

    # Keep only needed columns
    keep_cols = ['id', 'com_type', 'speaker', 'role', 'institution', 'president',
                 'date', 'year', 'month', 'source_file']

    # Add content variables that exist
    for var in CONTENT_VARS:
        if var in df.columns:
            keep_cols.append(var)
        else:
            df[var] = '.'
            keep_cols.append(var)

    return df[keep_cols]

def process_speeches(filepath):
    """Process Fed speeches."""
    print("\n" + "="*70)
    print("PROCESSING: SPEECHES")
    print("="*70)

    df = pd.read_csv(filepath)
    print(f"Loaded {len(df)} records")

    # Standardize speaker names (already have 'speaker' column)
    df['speaker'] = df['speaker'].apply(standardize_speaker_name)

    # Process dates
    df[['date', 'year', 'month']] = df['date'].apply(
        lambda x: pd.Series(process_date(x))
    )

    # Store original role field for institution determination
    df['role_original'] = df['role']

    # Determine roles
    df['role'] = df.apply(
        lambda row: determine_role(row['speaker'], row['role_original'], row['date'], 'speeches'),
        axis=1
    )

    # Determine institution
    df['institution'] = df.apply(
        lambda row: determine_institution(row['speaker'], row['role_original'], row['role']),
        axis=1
    )

    # Determine appointing president
    df['president'] = df.apply(
        lambda row: determine_appointing_president(row['speaker'], row['date'], row['role']),
        axis=1
    )

    # Rename url to source_url
    df = df.rename(columns={'url': 'source_url'})

    # Create standardized columns
    df['com_type'] = 'speeches'
    df['source_file'] = 'speeches_content.csv'

    # Keep only needed columns
    keep_cols = ['id', 'com_type', 'speaker', 'role', 'institution', 'president',
                 'date', 'year', 'month', 'source_file']

    # Add content variables that exist
    for var in CONTENT_VARS:
        if var in df.columns:
            keep_cols.append(var)
        else:
            df[var] = '.'
            keep_cols.append(var)

    return df[keep_cols]

def process_press_conferences(filepath):
    """Process press conferences."""
    print("\n" + "="*70)
    print("PROCESSING: PRESS CONFERENCES")
    print("="*70)

    df = pd.read_csv(filepath)
    print(f"Loaded {len(df)} records")

    # Standardize speaker names
    df['speaker'] = df['speaker'].apply(standardize_speaker_name)

    # Process dates
    df[['date', 'year', 'month']] = df['date'].apply(
        lambda x: pd.Series(process_date(x))
    )

    # Determine roles
    df['role'] = df.apply(
        lambda row: determine_role(row['speaker'], None, row['date'], 'presscon'),
        axis=1
    )

    # Determine institution (Board for Chair, blank for Other)
    df['institution'] = df.apply(
        lambda row: 'Board' if row['role'] == 'Chair' else '.',
        axis=1
    )

    # Determine appointing president
    df['president'] = df.apply(
        lambda row: determine_appointing_president(row['speaker'], row['date'], row['role']),
        axis=1
    )

    # Rename id column
    df = df.rename(columns={'press_conf_id': 'id'})

    # Create standardized columns
    df['com_type'] = 'presscon'
    df['source_file'] = 'press_conferences_content.csv'

    # Keep only needed columns
    keep_cols = ['id', 'com_type', 'speaker', 'role', 'institution', 'president',
                 'date', 'year', 'month', 'source_file']

    # Add content variables that exist
    for var in CONTENT_VARS:
        if var in df.columns:
            keep_cols.append(var)
        else:
            df[var] = '.'
            keep_cols.append(var)

    return df[keep_cols]

# ============================================================================
# MAIN EXECUTION
# ============================================================================

print("\n" + "="*70)
print("COMBINING FED COMMUNICATIONS DATA")
print("="*70)

# Create output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Process each source
dfs = []

if os.path.exists(INPUT_FILES['minutes']):
    dfs.append(process_minutes(INPUT_FILES['minutes']))
else:
    print(f"\nWarning: {INPUT_FILES['minutes']} not found")

if os.path.exists(INPUT_FILES['statements']):
    dfs.append(process_statements(INPUT_FILES['statements']))
else:
    print(f"\nWarning: {INPUT_FILES['statements']} not found")

if os.path.exists(INPUT_FILES['transcripts']):
    dfs.append(process_transcripts(INPUT_FILES['transcripts']))
else:
    print(f"\nWarning: {INPUT_FILES['transcripts']} not found")

if os.path.exists(INPUT_FILES['speeches']):
    dfs.append(process_speeches(INPUT_FILES['speeches']))
else:
    print(f"\nWarning: {INPUT_FILES['speeches']} not found")

if os.path.exists(INPUT_FILES['presscon']):
    dfs.append(process_press_conferences(INPUT_FILES['presscon']))
else:
    print(f"\nWarning: {INPUT_FILES['presscon']} not found")

# Combine all dataframes
print("\n" + "="*70)
print("COMBINING ALL SOURCES")
print("="*70)

if len(dfs) > 0:
    combined_df = pd.concat(dfs, ignore_index=True)

    # Sort by date
    combined_df['date_sort'] = pd.to_datetime(combined_df['date'], errors='coerce')
    combined_df = combined_df.sort_values('date_sort')
    combined_df = combined_df.drop('date_sort', axis=1)

    # Save combined dataset
    output_file = os.path.join(OUTPUT_DIR, 'Combined_Dataset.csv')
    combined_df.to_csv(output_file, index=False)

    print(f"\n✓ Combined dataset saved to: {output_file}")
    print(f"  Total records: {len(combined_df)}")
    print(f"  Shape: {combined_df.shape}")
    print(f"\n  Records by communication type:")
    print(combined_df['com_type'].value_counts().to_string())
    print(f"\n  Records by role:")
    print(combined_df['role'].value_counts().to_string())
    print(f"\n  Date range: {combined_df['date'].min()} to {combined_df['date'].max()}")

    # Display first few rows
    print(f"\n  Sample of first 5 rows:")
    print(combined_df[['id', 'com_type', 'speaker', 'role', 'institution', 'president',
                       'date', 'year', 'month']].head())

else:
    print("\nError: No data files found!")

print("\n" + "="*70)
print("PROCESSING COMPLETE!")
print("="*70)

Mounted at /content/drive

COMBINING FED COMMUNICATIONS DATA

PROCESSING: MINUTES
Loaded 199 records

PROCESSING: STATEMENTS
Loaded 198 records

PROCESSING: TRANSCRIPTS
Loaded 2589 records

PROCESSING: SPEECHES
Loaded 1121 records

PROCESSING: PRESS CONFERENCES
Loaded 170 records

COMBINING ALL SOURCES

✓ Combined dataset saved to: /content/drive/MyDrive/FedComs/SummaryStats/Combined_Dataset.csv
  Total records: 4277
  Shape: (4277, 85)

  Records by communication type:
com_type
transcripts    2589
speeches       1121
minutes         199
statements      198
presscon        170

  Records by role:
role
Regional President    1771
Governor              1548
Chair                  475
FOMC                   397
Other                   85
.                        1

  Date range: 2000-02-02 to 2025-09-23

  Sample of first 5 rows:
                                 id     com_type     speaker  \
0                  minutes_20000202      minutes        fomc   
397     EBoehne_20000202_transcrip

In [8]:
"""
Combine Fed Communications Data
Combines minutes, statements, transcripts, speeches, and press conferences
into a single dataset with standardized variables.
"""

from google.colab import drive
drive.mount("/content/drive", force_remount=True)

import pandas as pd
import numpy as np
from datetime import datetime
import os

# ============================================================================
# CONFIGURATION
# ============================================================================

BASE_DIR = '/content/drive/MyDrive/FedComs'
OUTPUT_DIR = f'{BASE_DIR}/SummaryStats'

# Input files
INPUT_FILES = {
    'minutes': f'{BASE_DIR}/Minutes/minutes_content.csv',
    'statements': f'{BASE_DIR}/Statements/statement_content.csv',
    'transcripts': f'{BASE_DIR}/Transcripts/transcripts_content.csv',
    'speeches': f'{BASE_DIR}/Speeches/speeches_content.csv',
    'presscon': f'{BASE_DIR}/PressConf/press_conferences_content.csv'
}

# ============================================================================
# SPEAKER NAME STANDARDIZATION
# ============================================================================

def standardize_speaker_name(name):
    """
    Convert full names to format: firstinitiallastname (e.g., 'jpowell')
    Handle various name formats and return '.' for missing values.
    """
    if pd.isna(name) or name == '' or str(name).strip() == '':
        return '.'

    name = str(name).strip()

    # Handle special cases
    if name.lower() in ['other', 'fomc', '.']:
        return name.lower() if name.lower() != '.' else '.'

    # Remove common titles and suffixes
    name = name.replace('Jr.', '').replace('Jr', '').replace('Sr.', '').replace('Sr', '')
    name = name.replace('Dr.', '').replace('Dr', '').replace('Mr.', '').replace('Ms.', '')

    # Split name into parts
    parts = name.split()

    if len(parts) == 0:
        return '.'
    elif len(parts) == 1:
        # Just last name
        return parts[0].lower()
    else:
        # First initial + last name
        first_initial = parts[0][0].lower()
        last_name = parts[-1].lower()
        return f"{first_initial}{last_name}"

# ============================================================================
# ROLE DETERMINATION
# ============================================================================

def determine_role(speaker, role_field, date, com_type):
    """
    Determine role: Chair, Governor, Regional President, FOMC, or Other
    """
    # For statements and minutes, always FOMC
    if com_type in ['statements', 'minutes']:
        return 'FOMC'

    # For press conferences with "Other"
    if com_type == 'presscon' and speaker == 'other':
        return 'Other'

    # If role field exists, use it
    if pd.notna(role_field) and role_field != '':
        role_str = str(role_field).strip()

        # Check for Chair
        if 'chair' in role_str.lower() and 'vice' not in role_str.lower():
            return 'Chair'

        # Check for Governor (including Vice Chair)
        if 'governor' in role_str.lower() or 'vice chair' in role_str.lower():
            return 'Governor'

        # Regional banks
        regional_banks = [
            'boston', 'new york', 'philadelphia', 'cleveland', 'richmond',
            'atlanta', 'chicago', 'st. louis', 'minneapolis', 'kansas city',
            'dallas', 'san francisco'
        ]

        if any(bank in role_str.lower() for bank in regional_banks):
            return 'Regional President'

    # Fallback: determine chair by date if speaker is standardized chair name
    if pd.notna(date):
        try:
            date_obj = pd.to_datetime(date)

            # Known Fed Chairs and their tenures
            if speaker in ['agreenspan', 'greenspan']:
                if date_obj < pd.to_datetime('2006-02-01'):
                    return 'Chair'
            elif speaker in ['bbernanke', 'bernanke']:
                if pd.to_datetime('2006-02-01') <= date_obj < pd.to_datetime('2014-02-01'):
                    return 'Chair'
            elif speaker in ['jyellen', 'yellen']:
                if pd.to_datetime('2014-02-01') <= date_obj < pd.to_datetime('2018-02-05'):
                    return 'Chair'
            elif speaker in ['jpowell', 'powell']:
                if date_obj >= pd.to_datetime('2018-02-05'):
                    return 'Chair'
        except:
            pass

    return '.'

# ============================================================================
# DATE PROCESSING
# ============================================================================

def process_date(date_val):
    """
    Convert date to standard format and extract year/month.
    Returns: (formatted_date, year, month) or ('.', '.', '.') if invalid
    """
    if pd.isna(date_val) or date_val == '' or str(date_val).strip() == '':
        return '.', '.', '.'

    try:
        # Parse date
        date_obj = pd.to_datetime(date_val, errors='coerce')

        if pd.isna(date_obj):
            return '.', '.', '.'

        # Format as YYYY-MM-DD
        formatted_date = date_obj.strftime('%Y-%m-%d')
        year = str(date_obj.year)
        month = str(date_obj.month)

        return formatted_date, year, month
    except:
        return '.', '.', '.'

# ============================================================================
# CONTENT VARIABLES - Define the 77 variables we want to keep
# ============================================================================

CONTENT_VARS = [
    'source_url',
    'sentences_on_labor',
    'sentences_on_inflation',
    'sentences_on_both',
    'total_sentences',
    'labor_share_of_labor_inflation_sentences',
    # Labor counts
    'labor_Employment_count',
    'labor_Unemployment_count',
    'labor_Participation_count',
    'labor_Wages_count',
    'labor_Vacancies_count',
    'labor_Quits_count',
    'labor_Layoffs_count',
    'labor_Hiring_count',
    # Inflation counts
    'inflation_Commodity_Prices_count',
    'inflation_Core_count',
    'inflation_Core_CPI_count',
    'inflation_Core_PCE_count',
    'inflation_Energy_count',
    'inflation_Food_count',
    'inflation_Goods_count',
    'inflation_Headline_count',
    'inflation_Headline_CPI_count',
    'inflation_Headline_PCE_count',
    'inflation_Housing_count',
    'inflation_Inflation_Expectations_count',
    'inflation_PPI_count',
    'inflation_Services_count',
    'inflation_Wage_Inflation_count',
    # Labor emphasis
    'labor_emphasis_Employment',
    'labor_emphasis_Unemployment',
    'labor_emphasis_Participation',
    'labor_emphasis_Wages',
    'labor_emphasis_Vacancies',
    'labor_emphasis_Quits',
    'labor_emphasis_Layoffs',
    'labor_emphasis_Hiring',
    # Inflation emphasis
    'inflation_emphasis_Commodity_Prices',
    'inflation_emphasis_Core',
    'inflation_emphasis_Core_CPI',
    'inflation_emphasis_Core_PCE',
    'inflation_emphasis_Energy',
    'inflation_emphasis_Food',
    'inflation_emphasis_Goods',
    'inflation_emphasis_Headline',
    'inflation_emphasis_Headline_CPI',
    'inflation_emphasis_Headline_PCE',
    'inflation_emphasis_Housing',
    'inflation_emphasis_Inflation_Expectations',
    'inflation_emphasis_PPI',
    'inflation_emphasis_Services',
    'inflation_emphasis_Wage_Inflation',
    # Labor share of total sentences
    'labor_share_total_sentences_Employment',
    'labor_share_total_sentences_Unemployment',
    'labor_share_total_sentences_Participation',
    'labor_share_total_sentences_Wages',
    'labor_share_total_sentences_Vacancies',
    'labor_share_total_sentences_Quits',
    'labor_share_total_sentences_Layoffs',
    'labor_share_total_sentences_Hiring',
    # Inflation share of total sentences
    'inflation_share_total_sentences_Commodity_Prices',
    'inflation_share_total_sentences_Core',
    'inflation_share_total_sentences_Core_CPI',
    'inflation_share_total_sentences_Core_PCE',
    'inflation_share_total_sentences_Energy',
    'inflation_share_total_sentences_Food',
    'inflation_share_total_sentences_Goods',
    'inflation_share_total_sentences_Headline',
    'inflation_share_total_sentences_Headline_CPI',
    'inflation_share_total_sentences_Headline_PCE',
    'inflation_share_total_sentences_Housing',
    'inflation_share_total_sentences_Inflation_Expectations',
    'inflation_share_total_sentences_PPI',
    'inflation_share_total_sentences_Services',
    'inflation_share_total_sentences_Wage_Inflation'
]

# ============================================================================
# PROCESS EACH DATA SOURCE
# ============================================================================

def process_minutes(filepath):
    """Process FOMC minutes."""
    print("\n" + "="*70)
    print("PROCESSING: MINUTES")
    print("="*70)

    df = pd.read_csv(filepath)
    print(f"Loaded {len(df)} records")

    # Create standardized columns
    df['com_type'] = 'minutes'
    df['speaker'] = 'fomc'
    df['role'] = 'FOMC'

    # Process dates
    df[['date', 'year', 'month']] = df['date'].apply(
        lambda x: pd.Series(process_date(x))
    )

    # Get source file name
    df['source_file'] = 'minutes_content.csv'

    # Keep only needed columns
    keep_cols = ['id', 'com_type', 'speaker', 'role', 'date', 'year', 'month', 'source_file']

    # Add content variables that exist
    for var in CONTENT_VARS:
        if var in df.columns:
            keep_cols.append(var)
        else:
            df[var] = '.'
            keep_cols.append(var)

    return df[keep_cols]

def process_statements(filepath):
    """Process FOMC statements."""
    print("\n" + "="*70)
    print("PROCESSING: STATEMENTS")
    print("="*70)

    df = pd.read_csv(filepath)
    print(f"Loaded {len(df)} records")

    # Create standardized columns
    df['com_type'] = 'statements'
    df['speaker'] = 'fomc'
    df['role'] = 'FOMC'
    df['source_url'] = '.'  # Statements don't have source_url

    # Process dates
    df[['date', 'year', 'month']] = df['date'].apply(
        lambda x: pd.Series(process_date(x))
    )

    # Get source file name
    df['source_file'] = 'statement_content.csv'

    # Keep only needed columns
    keep_cols = ['id', 'com_type', 'speaker', 'role', 'date', 'year', 'month', 'source_file']

    # Add content variables that exist
    for var in CONTENT_VARS:
        if var in df.columns:
            keep_cols.append(var)
        else:
            df[var] = '.'
            keep_cols.append(var)

    return df[keep_cols]

def process_transcripts(filepath):
    """Process FOMC transcripts."""
    print("\n" + "="*70)
    print("PROCESSING: TRANSCRIPTS")
    print("="*70)

    df = pd.read_csv(filepath)
    print(f"Loaded {len(df)} records")

    # Standardize speaker names
    df['speaker'] = df['official_name'].apply(standardize_speaker_name)

    # Process dates
    df[['date', 'year', 'month']] = df['date'].apply(
        lambda x: pd.Series(process_date(x))
    )

    # Determine roles
    df['role'] = df.apply(
        lambda row: determine_role(row['speaker'], row['role'], row['date'], 'transcripts'),
        axis=1
    )

    # Rename id column
    df = df.rename(columns={'transcript_id': 'id'})

    # Create standardized columns
    df['com_type'] = 'transcripts'
    df['source_url'] = '.'  # Transcripts don't have source_url
    df['source_file'] = 'transcripts_content.csv'

    # Keep only needed columns
    keep_cols = ['id', 'com_type', 'speaker', 'role', 'date', 'year', 'month', 'source_file']

    # Add content variables that exist
    for var in CONTENT_VARS:
        if var in df.columns:
            keep_cols.append(var)
        else:
            df[var] = '.'
            keep_cols.append(var)

    return df[keep_cols]

def process_speeches(filepath):
    """Process Fed speeches."""
    print("\n" + "="*70)
    print("PROCESSING: SPEECHES")
    print("="*70)

    df = pd.read_csv(filepath)
    print(f"Loaded {len(df)} records")

    # Standardize speaker names (already have 'speaker' column)
    df['speaker'] = df['speaker'].apply(standardize_speaker_name)

    # Process dates
    df[['date', 'year', 'month']] = df['date'].apply(
        lambda x: pd.Series(process_date(x))
    )

    # Determine roles
    df['role'] = df.apply(
        lambda row: determine_role(row['speaker'], row['role'], row['date'], 'speeches'),
        axis=1
    )

    # Rename url to source_url
    df = df.rename(columns={'url': 'source_url'})

    # Create standardized columns
    df['com_type'] = 'speeches'
    df['source_file'] = 'speeches_content.csv'

    # Keep only needed columns
    keep_cols = ['id', 'com_type', 'speaker', 'role', 'date', 'year', 'month', 'source_file']

    # Add content variables that exist
    for var in CONTENT_VARS:
        if var in df.columns:
            keep_cols.append(var)
        else:
            df[var] = '.'
            keep_cols.append(var)

    return df[keep_cols]

def process_press_conferences(filepath):
    """Process press conferences."""
    print("\n" + "="*70)
    print("PROCESSING: PRESS CONFERENCES")
    print("="*70)

    df = pd.read_csv(filepath)
    print(f"Loaded {len(df)} records")

    # Standardize speaker names
    df['speaker'] = df['speaker'].apply(standardize_speaker_name)

    # Process dates
    df[['date', 'year', 'month']] = df['date'].apply(
        lambda x: pd.Series(process_date(x))
    )

    # Determine roles
    df['role'] = df.apply(
        lambda row: determine_role(row['speaker'], None, row['date'], 'presscon'),
        axis=1
    )

    # Rename id column
    df = df.rename(columns={'press_conf_id': 'id'})

    # Create standardized columns
    df['com_type'] = 'presscon'
    df['source_file'] = 'press_conferences_content.csv'

    # Keep only needed columns
    keep_cols = ['id', 'com_type', 'speaker', 'role', 'date', 'year', 'month', 'source_file']

    # Add content variables that exist
    for var in CONTENT_VARS:
        if var in df.columns:
            keep_cols.append(var)
        else:
            df[var] = '.'
            keep_cols.append(var)

    return df[keep_cols]

# ============================================================================
# MAIN EXECUTION
# ============================================================================

print("\n" + "="*70)
print("COMBINING FED COMMUNICATIONS DATA")
print("="*70)

# Create output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Process each source
dfs = []

if os.path.exists(INPUT_FILES['minutes']):
    dfs.append(process_minutes(INPUT_FILES['minutes']))
else:
    print(f"\nWarning: {INPUT_FILES['minutes']} not found")

if os.path.exists(INPUT_FILES['statements']):
    dfs.append(process_statements(INPUT_FILES['statements']))
else:
    print(f"\nWarning: {INPUT_FILES['statements']} not found")

if os.path.exists(INPUT_FILES['transcripts']):
    dfs.append(process_transcripts(INPUT_FILES['transcripts']))
else:
    print(f"\nWarning: {INPUT_FILES['transcripts']} not found")

if os.path.exists(INPUT_FILES['speeches']):
    dfs.append(process_speeches(INPUT_FILES['speeches']))
else:
    print(f"\nWarning: {INPUT_FILES['speeches']} not found")

if os.path.exists(INPUT_FILES['presscon']):
    dfs.append(process_press_conferences(INPUT_FILES['presscon']))
else:
    print(f"\nWarning: {INPUT_FILES['presscon']} not found")

# Combine all dataframes
print("\n" + "="*70)
print("COMBINING ALL SOURCES")
print("="*70)

if len(dfs) > 0:
    combined_df = pd.concat(dfs, ignore_index=True)

    # Sort by date
    combined_df['date_sort'] = pd.to_datetime(combined_df['date'], errors='coerce')
    combined_df = combined_df.sort_values('date_sort')
    combined_df = combined_df.drop('date_sort', axis=1)

    # Save combined dataset
    output_file = os.path.join(OUTPUT_DIR, 'Combined_Dataset.csv')
    combined_df.to_csv(output_file, index=False)

    print(f"\n✓ Combined dataset saved to: {output_file}")
    print(f"  Total records: {len(combined_df)}")
    print(f"  Shape: {combined_df.shape}")
    print(f"\n  Records by communication type:")
    print(combined_df['com_type'].value_counts().to_string())
    print(f"\n  Records by role:")
    print(combined_df['role'].value_counts().to_string())
    print(f"\n  Date range: {combined_df['date'].min()} to {combined_df['date'].max()}")

    # Display first few rows
    print(f"\n  Sample of first 5 rows:")
    print(combined_df[['id', 'com_type', 'speaker', 'role', 'date', 'year', 'month']].head())

else:
    print("\nError: No data files found!")

print("\n" + "="*70)
print("PROCESSING COMPLETE!")
print("="*70)

Mounted at /content/drive

COMBINING FED COMMUNICATIONS DATA

PROCESSING: MINUTES
Loaded 199 records

PROCESSING: STATEMENTS
Loaded 198 records

PROCESSING: TRANSCRIPTS
Loaded 2589 records

PROCESSING: SPEECHES
Loaded 1121 records

PROCESSING: PRESS CONFERENCES
Loaded 170 records

COMBINING ALL SOURCES

✓ Combined dataset saved to: /content/drive/MyDrive/FedComs/SummaryStats/Combined_Dataset.csv
  Total records: 4277
  Shape: (4277, 83)

  Records by communication type:
com_type
transcripts    2589
speeches       1121
minutes         199
statements      198
presscon        170

  Records by role:
role
Regional President    1771
Governor              1548
Chair                  475
FOMC                   397
Other                   85
.                        1

  Date range: 2000-02-02 to 2025-09-23

  Sample of first 5 rows:
                                 id     com_type     speaker  \
0                  minutes_20000202      minutes        fomc   
397     EBoehne_20000202_transcrip